In [4]:
# Data Extraction
import pandas as pd 
import numpy as np
factors = pd.read_excel('./midterm_2_data.xlsx', sheet_name = 'factors (excess returns)').set_index('Date')
portfolio = pd.read_excel('./midterm_2_data.xlsx', sheet_name = 'portfolios (excess returns)').set_index('Date')

annualization_factor = 12

import statsmodels.api as sm
def time_series_regression(portfolio, factors, annualization_factor, FF3F = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] * annualization_factor
        ff_report.loc[portf, 'beta_mkt'] = res.params[1]
        if FF3F:
            ff_report.loc[portf, 'Size beta'] = res.params[2] 
            ff_report.loc[portf, 'Value beta'] = res.params[3]
            
        ff_report.loc[portf, 'R-squared'] = res.rsquared

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report


In [5]:
time_series_regression(portfolio, factors, annualization_factor)

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_13543/972016806.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'beta_mkt'] = res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_13543/972016806.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'beta_mkt'] = res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_13543/972016806.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

,alpha_hat,beta_mkt,R-squared
NoDur,0.029253,0.739522,0.617919
Durbl,0.010734,1.271865,0.613493
Manuf,-0.000996,1.049482,0.870268
Enrgy,-0.015117,0.992222,0.465602
HiTec,0.028207,1.154959,0.829498
Telcm,0.003506,0.837326,0.588052
Shops,0.026739,0.946928,0.742161
Hlth,0.031862,0.757605,0.580514
Utils,0.013710,0.527879,0.342654
Other,-0.019780,1.115433,0.910098


In [28]:
def calc_pricing_regression(rets, factors, annualization_factor, intercept=True):
    # summary to store the betas and model statistics 
    
    if intercept:
        factors = sm.add_constant(factors)
    summary = {f'{k} Beta': [] for k in factors.columns[1:]}
    summary['Alpha']  = []
    summary['R Square']    = []
    for asset in rets.columns:
        model = sm.OLS(rets[asset], factors).fit()
        for k, _ in summary.items():
            try:
                summary[k].append(model.params[k[:-5]])
            except:
                pass
        summary['R Square'].append(model.rsquared)
        summary['Alpha'].append(model.params[0] * annualization_factor)
            
    return pd.DataFrame(summary, index=rets.columns)

def calc_cross_sectional_regression(mean_rets, betas, intercept=False):
    # dependent variable = mean returns of the portfolios 
    # independent variables = betas obtained from the t-s regressions (these betas are from each regression of the aset against the factors)
    if intercept:
        betas = sm.add_constant(betas)
    model = sm.OLS(mean_rets, betas).fit()
    params = model.params.to_frame('Cross Sectional Regression')
    params.loc['R Square'] = model.rsquared
    params.loc['MAE'] = model.resid.abs().mean() 
    return params

frames = []

ts_premia = portfolio.mean() * annualization_factor

pricing_regr = calc_pricing_regression(portfolio, factors, annualization_factor, intercept=False)
summary_cs = calc_cross_sectional_regression(ts_premia, pricing_regr.iloc[:, :-2], intercept=False)
summary_cs

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_13543/2144895192.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary['Alpha'].append(model.params[0] * annualization_factor)


,Cross Sectional Regression
HML Beta,0.099315
UMD Beta,-0.247383
R Square,0.215801
MAE,0.070022


In [29]:
factors

,MKT,HML,UMD
Date,,,
1980-01-31,0.0551,0.0175,0.0755
1980-02-29,-0.0122,0.0061,0.0788
1980-03-31,-0.1290,-0.0101,-0.0955
1980-04-30,0.0397,0.0106,-0.0043
1980-05-31,0.0526,0.0038,-0.0112
...,...,...,...
2023-04-30,0.0061,-0.0004,0.0162
2023-05-31,0.0035,-0.0772,-0.0063
2023-06-30,0.0646,-0.0026,-0.0237
